### 准备工作

In [1]:
from pymongo import MongoClient
import pymongo

### 连接MongoDB

In [2]:
client = MongoClient(host="127.0.0.1",port=27017)
# client = MongoClient("mongodb://localhost:27017/")

### 指定数据库

In [3]:
db = client.newDB
# client["newDB"]

### 指定集合

In [4]:
collection = db.info
# collection = db["info"]

在MongoDB中，每条数据其实都有一个_id属性来唯一标识。如果没有显式指明该属性，MongoDB会自动产生一个ObjectId类型的_id属性。insert()方法会在执行后返回_id值。

### 插入数据

In [22]:
student = {
    'id':'20170101',
    'name':'John',
    'age':20,
    'gender':'male'
}

在PyMongo 3.x版本中，官方已经不推荐使用insert()方法了。当然，继续使用也没有什么问题。官方推荐使用insert_one()和insert_many()方法来分别插入单条记录和多条记录

In [23]:
result = collection.insert_one(student)
print(result)

In [24]:
student1 = {
    'id':'20170102',
    'name':'Jordan',
    'age':20,
    'gender':'male'
}
student2 = {
    'id':'201701202',
    'name':'Mike',
    'age':21,
    'gender':'male'
}

In [25]:
result = collection.insert_many([student1,student2])
print(result)

In [26]:
print(result.inserted_ids)

[ObjectId('5dd2add38ae2dfb6b4a3e567'), ObjectId('5dd2add38ae2dfb6b4a3e568')]


![](./pic/insert1.PNG)

### 查询

In [29]:
result = collection.find_one({'name':'Mike'})

可以发现，它多了_id属性，这就是MongoDB在插入过程中自动添加的。

In [30]:
print(type(result))
print(result)

<class 'dict'>
{'_id': ObjectId('5dd2add38ae2dfb6b4a3e568'), 'id': '201701202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


In [34]:
results = collection.find({'age':20})
print(result)

{'_id': ObjectId('5dd2add38ae2dfb6b4a3e568'), 'id': '201701202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


In [35]:
for result in results:
    print(result)

{'_id': ObjectId('5dd2add08ae2dfb6b4a3e566'), 'id': '20170101', 'name': 'John', 'age': 20, 'gender': 'male'}
{'_id': ObjectId('5dd2add38ae2dfb6b4a3e567'), 'id': '20170102', 'name': 'Jordan', 'age': 20, 'gender': 'male'}


In [39]:
results = collection.find({'name':{'$regex':'^M.*'}})

In [40]:
print(results)

![](./pic/re.PNG)

In [41]:
for result in results:
    print(result)

{'_id': ObjectId('5dd2add38ae2dfb6b4a3e568'), 'id': '201701202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


### 计数

In [46]:
count = collection.find().count()
# count = collection.count_documents
print(count)

3


D:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


### 排序
排序时，直接调用sort()方法，并在其中传入排序的字段及升降序标志即可

In [49]:
results = collection.find().sort('name',pymongo.ASCENDING)

In [50]:
print([result['name'] for result in results])

['John', 'Jordan', 'Mike']


### 更新
这里我们要更新name为Kevin的数据的年龄：首先指定查询条件，然后将数据查询出来，修改年龄后调用update()方法将原条件和修改后的数据传入。

In [5]:
condition = {'name':'John'}

In [6]:
student = collection.find_one(condition)

In [7]:
print(student)

{'_id': ObjectId('5dd2add08ae2dfb6b4a3e566'), 'id': '20170101', 'name': 'John', 'age': 25, 'gender': 'male'}


In [8]:
student['age'] = 25

In [9]:
result = collection.update(condition,student)
print(result)

{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}


D:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.


![](./pic/update1.PNG)

另外，我们也可以使用$set操作符对数据进行更新，代码如下：

In [10]:
student['age'] = 26

In [11]:
result = collection.update(condition,{'$set':student})

D:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.


In [12]:
result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

![](./pic/update2.PNG)

另外，update()方法其实也是官方不推荐使用的方法。这里也分为update_one()方法和update_many()方法，用法更加严格，它们的第二个参数需要使用$类型操作符作为字典的键名，示例如下：

In [13]:
condition = {"name":"Mike"}

In [14]:
student = collection.find_one(condition)
print(student)

{'_id': ObjectId('5dd2add38ae2dfb6b4a3e568'), 'id': '201701202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


In [22]:
student["age"] = 27

这里调用了update_one()方法，第二个参数不能再直接传入修改后的字典，而是需要使用{'$set': student}这样的形式，其返回结果是UpdateResult类型。然后分别调用matched_count和modified_count属性，可以获得匹配的数据条数和影响的数据条数。

In [23]:
result = collection.update_one(condition,{'$set':student})
print(result)
print(result.matched_count,result.modified_count)

1 1


In [24]:
result = collection.update_one(condition,{'$set':student})
print(result)
print(result.matched_count,result.modified_count)

1 0


![](./pic/update3.PNG)

In [25]:
condition = {"age":{"$gt":25}}

在下面这条语句没有执行之前： 
![](update4.PNG)

In [29]:
results = collection.update_one(condition,{"$inc":{"age":1}})
print(results)

![](./pic/update5.PNG)

In [31]:
print(results.matched_count, results.modified_count) 

1 1


### 删除

In [32]:
result = collection.remove({"name":"Mike"})
print(result)

{'n': 1, 'ok': 1.0}


D:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


![](./pic/delete1.PNG)

另外，这里依然存在两个新的推荐方法——delete_one()和delete_many()。示例如下:  
因为info中数据不够，所以...只把操作写在这
result = collection.delete_one({'name': 'Kevin'})  
print(result)  
print(result.deleted_count)  
result = collection.delete_many({'age': {'$lt': 25}})  
print(result.deleted_count) 